In [4]:
#clean environment
from IPython import get_ipython
get_ipython().magic('reset -f')
from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import json
import requests
import numpy as np
from datetime import datetime
from pandas.tseries.offsets import MonthEnd


'''
ask token
'''    
url = "https://login-p10.xiaoshouyi.com/auc/oauth2/token"
payload = {
    "grant_type": "password",
    "client_id": "351fb11916fc85504ba5f6ec3b96988b",
    "client_secret": "8f9355878504005ba782ca049d013d4e",
    "redirect_uri": "https://crm-p10.xiaoshouyi.com",
    "username": "11020964@twkd.com",
    "password": "Kd0627ubGHagpQ"
}
response = requests.post(url, data=payload)
content = response.json()
ac_token = content["access_token"]

'''
select from account
'''
#for department in unique_departments:
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
account_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT 
                        accountCode__c as 公司代號, 
                        accountName as 公司全名, 
                        customItem296__c as 負責人,
                        customItem317__c as 公司建立日期, 
                        SAP_CompanyID__c as SAP公司代號, 
                        customItem253__c as 資料區域, 
                        customItem199__c as 公司型態
                    FROM 
                        account
                    WHERE 
                        customItem253__c LIKE 'TW-%' 
                        AND customItem253__c NOT IN ('TW-888', 'TW-INV')
                        AND customItem317__c IS NOT NULL 
                        AND SAP_CompanyID__c IS NOT NULL
                        AND customItem199__c NOT in ('X', 'EZ', 'H', 'I', 'SE', 'W', 'P', 'L', 'FZ', 'GZ', 'KZ', 'SZ', 'Z')
                   ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    account_df = pd.concat([account_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']


'''
select from related_contact
'''
url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
headers = {
    "Authorization": f"Bearer {ac_token}",
    "Content-Type":"application/x-www-form-urlencoded"
    # Replace with your actual access token
}
queryLocator = ''
contact_df = pd.DataFrame()
 
while True:
    data = {
        "xoql": '''SELECT id, customItem8__c as 公司代號, name as 關係聯絡人代號,
                   contactCode__c__c as 聯絡人代號, customItem2__c.contactName as 負責人, contactPhone__c__c as 負責人手機號碼
                   FROM customEntity22__c where dimDepart.departName like 'TW-%'
                          ''',
        "batchCount": 2000,
        "queryLocator": queryLocator
    }
    response = requests.post(url_2, headers=headers, data=data)
    crm = response.json()
    data = pd.DataFrame(crm["data"]["records"])
    contact_df = pd.concat([contact_df, data], ignore_index=True, sort=False)
    
    if not crm['queryLocator']:
        break
    queryLocator = crm['queryLocator']

# =============================================================================
# '''
# select from contact
# '''
# #for department in unique_departments:
# url_2 = "https://api-p10.xiaoshouyi.com/rest/data/v2.0/query/xoqlScroll"
# headers = {
#     "Authorization": f"Bearer {ac_token}",
#     "Content-Type":"application/x-www-form-urlencoded"
#     # Replace with your actual access token
# }
# queryLocator = ''
# contact_df = pd.DataFrame()
#  
# while True:
#     data = {
#         "xoql": '''SELECT 
#                         accountCode__c as 公司代號,
#                         contactName as 負責人,
#                         mobile as 負責人手機號碼
#                     FROM 
#                         contact
#                     WHERE 
#                         dimDepart.departName LIKE 'TW-%'    
#                    ''',
#         "batchCount": 2000,
#         "queryLocator": queryLocator
#     }
#     response = requests.post(url_2, headers=headers, data=data)
#     crm = response.json()
#     data = pd.DataFrame(crm["data"]["records"])
#     contact_df = pd.concat([contact_df, data], ignore_index=True, sort=False)
#     
#     if not crm['queryLocator']:
#         break
#     queryLocator = crm['queryLocator']
# =============================================================================
    
account_df = pd.merge(account_df, contact_df, on = ['公司代號', '負責人'], how = 'left')
account_df = account_df.fillna('')
# '''
# connect to SAP for restrict
# '''
# connection_string = "Driver={Devart ODBC Driver for ASE};Database=PRD;Server=192.168.1.211;UID=sapsa;PWD=kd29003039;Port=4901;String Types=Unicode"
# connection = pyodbc.connect(connection_string)
# sql ='''SELECT SAPSR3.KNA1.KUNNR ,
#                   SAPSR3.TKUKT.VTEXT
#                       FROM SAPSR3.KNA1
#                       LEFT JOIN SAPSR3.TKUKT
#                       ON SAPSR3.KNA1.KUKLA=SAPSR3.TKUKT.KUKLA
#                       AND SAPSR3.TKUKT.MANDT ='800'
#                       AND SAPSR3.TKUKT.SPRAS = 'M'
#     '''
# df2 = pd.read_sql_query(sql, connection)
# connection.close()
# df2 = df2.dropna(subset=['VTEXT'])
# df2.columns = ["SAP公司代號", "SAP信用管制"]
df2 = pd.DataFrame()
df2['SAP公司代號'] = ""
'''
no restrict
'''
no_restrict = pd.merge(account_df, df2, on = 'SAP公司代號', how = 'left')
no_restrict = no_restrict[no_restrict['SAP信用管制']!='管制']
no_restrict['負責人'] = no_restrict['負責人'].str.strip()
no_restrict['公司建立日期'] = no_restrict['公司建立日期'].astype(str)
# Replace '年' with '-', '月' with '-', and remove '日'
no_restrict['公司建立日期'] = no_restrict['公司建立日期'].str.replace('年', '-').str.replace('月', '-').str.replace('日', '').str.replace('號', '')
no_restrict['公司建立日期'] = no_restrict['公司建立日期'].str.replace('民國', '')
is_all_digits = no_restrict['公司建立日期'].str.isdigit()

# Update the format for all-digit values based on their length
no_restrict.loc[is_all_digits, '公司建立日期'] = no_restrict.loc[is_all_digits, '公司建立日期'].apply(
    lambda x: f'{x[:2]}-{x[2:4]}-{x[4:]}' if len(x) == 6 else f'{x[:3]}-{x[3:5]}-{x[5:]}' if len(x) == 7 else f'{x[:4]}-{x[4:6]}-{x[6:]}' if len(x) == 8 else x
)
no_restrict['公司建立日期'] = no_restrict['公司建立日期'].str.strip().replace('[/\.]', '-', regex=True)



'''
deal with date to make a new one
'''
no_restrict[['year', 'month', 'day']] = no_restrict['公司建立日期'].str.split('-', expand=True)
no_restrict['year'] = no_restrict['year'].str.lstrip('0')
is_three_digits = no_restrict['year'].str.len() <= 3
no_restrict.loc[is_three_digits, 'year'] = (1911 + no_restrict.loc[is_three_digits, 'year'].astype(int)).astype(str)
is_non_digit = ~no_restrict['year'].str.isdigit()
no_restrict = no_restrict[~is_non_digit]
no_restrict['year'] = no_restrict['year'].fillna('')
no_restrict['month'] = no_restrict['month'].fillna('')
no_restrict['day'] = no_restrict['day'].fillna('')
no_restrict['month'] = no_restrict['month'].apply(
    lambda x: f'0{x}' if len(x) == 1 else x
)
no_restrict['day'] = no_restrict['day'].apply(
    lambda x: f'0{x}' if len(x) == 1 else x
)
# Concatenate 'year', 'month', and 'day' into 'new_date'
no_restrict['new_date'] = no_restrict[['year', 'month', 'day']].agg('-'.join, axis=1)
no_restrict['new_date'] = no_restrict['new_date'].str.replace('月', '').str.replace('日', '')
#no_restrict.to_excel('nonono.xlsx', index = False)

'''
find the difference of years
'''
from datetime import datetime
current_month = datetime.now().month
next_month = (current_month % 12) + 1
current_date = datetime.now()
next_month_end = (current_date.replace(day=1) + MonthEnd(2))
specific_date = next_month_end 
#no_restrict.to_excel('03.xlsx', index = False)
def modify_date(date_str):
    if date_str.endswith('--') and len(date_str) == 9:
        year = str(1911 + int(date_str[:3]))  # Convert YY to YYYY (assuming 20th century)
        modified_date = year + '-' + date_str[3:5] + '-' + date_str[5:7]
        return modified_date
    else:
        return date_str
no_restrict['new_date'] = no_restrict['new_date'].apply(modify_date)
# Format the date as a string with the year as '112'
no_restrict['new_date'] = pd.to_datetime(no_restrict['new_date'])
specific_date_str = specific_date.strftime('%Y-%m-%d')
# Calculate the difference
no_restrict['周年'] = (specific_date - no_restrict['new_date']).dt.days // 365
valid_years = set(range(5, 101, 5))
no_restrict = no_restrict[no_restrict['周年'].isin(valid_years)]
no_restrict['訊息'] = no_restrict.apply(lambda row: f"您好~恭賀貴公司成立 {row['周年']} 周年，祝福貴司業務發展蒸蒸日上，在新的起點上更上一層樓，在新的旅途上更加輝煌，也感謝您一直對科定的支持與愛護，期待與您共創未來，共享雙贏!", axis=1)
no_restrict['是否發送周年禮'] = ''
no_restrict['實際發送人員'] = ''
no_restrict['備註'] = ''
no_restrict = no_restrict[no_restrict['負責人手機號碼']!='']
no_restrict = no_restrict.drop_duplicates(subset=['公司代號'], keep='first')
no_restrict = no_restrict.drop_duplicates(subset=['聯絡人代號'], keep='first')

next_month_str = '{:02d}'.format(next_month)
no_restrict = no_restrict[no_restrict['month']==next_month_str] 
no_restrict.drop_duplicates(subset=['公司代號'], keep='first', inplace=True)
no_restrict.to_excel(f'公司周年慶資料2025_{next_month_str}.xlsx', index = False)



<>:180: SyntaxWarning: invalid escape sequence '\.'
<>:180: SyntaxWarning: invalid escape sequence '\.'
C:\Users\11020964\AppData\Local\Temp\ipykernel_19476\2474483904.py:180: SyntaxWarning: invalid escape sequence '\.'
  no_restrict['公司建立日期'] = no_restrict['公司建立日期'].str.strip().replace('[/\.]', '-', regex=True)
C:\Users\11020964\AppData\Local\Temp\ipykernel_19476\2474483904.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')
C:\Users\11020964\AppData\Local\Temp\ipykernel_19476\2474483904.py:180: SyntaxWarning: invalid escape sequence '\.'
  no_restrict['公司建立日期'] = no_restrict['公司建立日期'].str.strip().replace('[/\.]', '-', regex=True)


KeyError: 'SAP信用管制'